In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sqlalchemy import create_engine, text
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns

#import nltk
#nltk.download('vader_lexicon')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

## Column definitions

- **Chat Id**: unique id given to a chat (good for indexing)
  
- **Picked Up On**: datetime of when Visitor initially responds to welcome chat (default '12/31/1899 9:00:00 PM' if never occurs)
  
- **Created On**: datetime when Visitor opens chat

- **Ended On**: datetime when Chat is ended (inactivity or closed by representative)

- **Location**: location of Visitor (when available)

- **Referrer**: The TEMO website page in which the user accesses the chat (hyperlink)
- **Lead Id**: If assigned lead type, unique lead id created for database purposes
- **Lead Type Id**: Type of lead
     - 0: not assigned a lead type
     - 1: Sales
     - 2: Service
     - 3: Other
- **Lead Type Name**: explained above
- **Original Referrer**: Website Page that lead them to the Referrer (helpful for ad campaign tracking)
- **Landing Referrer**: Same as Original Referrer except more complete data
- **Transcript Text**: transcript of the chat
- **Visitor Id**: Unique ID given to visitor (track if visitor returns)

In [ ]:
#read data from csv
df_raw = pd.read_csv('/home/apm204/cs210/Final Project/TEMO_Sunrooms_Blazeo_Chats.csv')
#df_raw.describe()
#df_raw.dtypes

df_path = r'/home/apm204/cs210/Final Project/raw_date_temo_sunrooms.xlsx'
df_raw.to_excel(df_path, index=False)

#remove redundant, useless information that is repeated throughout entire dataset
df_raw.drop(['Company Id', 'Company Name', 'Company Key'], axis=1, inplace=True)

## df_proc: dataframe used to show progression of processed data
df_proc = df_raw.copy()

# I. Begin Processing Data


**Step 1**: Create numerical column 'visitor_engaged' based on the categorical 'Picked Up On' column.

In [ ]:
try:

    #creating simpler columns to reflect activity of visitor in the chat
    df_proc['visitor_engaged'] = (df_proc['Picked Up On'] != '12/31/1899 9:00:00 PM').astype(int)

    """
    visitor_engaged:
        - 1 is visitor engaged in chat
        - 0 if visitor never engages in chat
    """

    #df_path = r'/home/apm204/cs210/Final Project/checker.csv'
    #df_proc[['Picked Up On', 'visitor_engaged']].to_csv(df_path, index=False)
    df_proc.drop(['Picked Up On'], axis=1, inplace=True)

except KeyError:
    print("\nPreventing KeyError from running cells out of order (all good)\n")

**Step 2**: Drop 'Lead Type Name' as it is the same as 'Lead Type Id'

In [ ]:
try:
    #Lead Type Name is just a label for Lead Type Id which provides chance for numeric analysis
    #Both provide same information, dropping Lead Type Name
    df_proc.drop(['Lead Type Name'], axis=1, inplace=True)
except KeyError:
    print("\nPreventing KeyError from running cells out of order (all good)\n")


**Step 3**: Advised by TEMO Sunrooms Representative that 'Original Referrer' possesses less important information when compared to 'Landing Referrer'. Dropping 'Original Referrer' as it will become negligible and difficult to run analysis since it is varied and categorical.

In [ ]:
try:
    #After discussing with TEMO Representative that provided access,
    #Original Referrer provides little value and is a subset of the data in Landing Referrer
    #Removing Original Referrer will maintain the information we have since it is contained in Landing Referrer
        #this also helps minimize the number of na records that need to be dealth with
    df_proc.drop(['Original Referrer'], axis=1, inplace=True)
except KeyError:
    print("\nPreventing KeyError from running cells out of order (all good)\n")

**Step 4**: Fill in na for 'Landing Referrer' and 'Referrer'

In [ ]:
#find the % of rows in 'Landing Referrer' that are na
print(f"{len(df_proc[df_proc['Landing Referrer'].isna()])/len(df_proc)*100:.2f}% of rows in 'Landing Referrer' are na.")

#find the % of visitors that have no landing referrer that were inactive
avg_nr_inactive = df_proc[df_proc['Landing Referrer'].isna()]['visitor_engaged'].mean()
print(f'{avg_nr_inactive*100:.2f}% of visitors with no landing referrer were engaged in chat')
    # ~80%
    #this is a large portion of active visitors therefore it would be helpful to fill these na values rather than delete them



#create dataframe, without rows that have na in 'Landing Referrer'
df_lr = df_proc.copy()
df_lr = df_lr[df_lr['Landing Referrer'].notna()]

#group by different referrers to see if the mode is significant enough to be used to fill in the rows that are na
referrers = df_lr.groupby('Landing Referrer').size()
print(f'\nThe largest referrer ({referrers.idxmax()}) covers {referrers.max()/len(df_lr)*100:.2f}% of all non-na rows.')
print(f'The second-largest referrer ({  referrers.nlargest(2).index[1]  }) covers {referrers.nlargest(2).iloc[1] / len(df_lr) * 100:.2f}% of all non-na rows.')
#google home page accounts for 40% of the referrers
    #second place (temo home page) accounts for less than 10%
#this sizable portion of the data means that we can fill na rows with google's home page

#since ~15% of rows are na, this will make google home page account for 55% of all landing referrers
mode_referrer = referrers.idxmax()
df_proc.loc[df_proc['Landing Referrer'].isna(), 'Landing Referrer'] = mode_referrer


#the next step in cleaning/processing this dataset is dealing with the rows with na in 'Referrer'
#this tends to go hand-in-hand with the chats that have a 'Visitor Id' of 0
#we will approach these two columns together

#begin by analyzing the similarities

#dataset of chats with no visitor id (id = 0)
df_visitor_id_na = df_proc.copy()
df_visitor_id_na = df_visitor_id_na[df_visitor_id_na['Visitor Id']==0]

print(f'length of df_visitor_id_na: {len(df_visitor_id_na)}')
df_visitor_id_na.head(5)

#df_path = r'/home/apm204/cs210/Final Project/vis_id_0.csv'
#df_visitor_id_na.to_csv(df_path, index=False)


#dataset of chats with no referrer (referrer = na)
df_refer_na = df_proc.copy()
df_refer_na = df_refer_na[df_refer_na['Referrer'].isna()]

print(f'length of df_refer_na: {len(df_refer_na)}')
df_refer_na.head(5)

#df_path = r'/home/apm204/cs210/Final Project/refer_na.csv'
#df_refer_na.to_csv(df_path, index=False)


#There is plenty of overlap between these datasets

#it can be seen that in both datasets there are subsequent rows with the same location;
#after inspection, these are the same chatters therefore they must be given the same visitor id

#since each location has one cluster of chats, we can assign a visitor id to each location cluster

#in order to fill the 'Referrer' column, we can first analyze the current makeup of the 'Referrer' column in the raw dataset

#create dataframe, without rows that have na in 'Landing Referrer'
df_ref = df_proc.copy()
df_ref = df_ref[df_ref['Referrer'].notna()]

#group by different referrers to see if the mode is significant enough to be used to fill in the rows that are na
referrers = df_ref.groupby('Referrer').size()
print(f'\nThe largest referrer ({referrers.idxmax()}) covers {referrers.max()/len(df_ref)*100:.2f}% of all non-na rows.')
print(f'The second-largest referrer ({  referrers.nlargest(2).index[1]  }) covers {referrers.nlargest(2).iloc[1] / len(df_ref) * 100:.2f}% of all non-na rows.')
print(f'The third-largest referrer ({  referrers.nlargest(3).index[2]  }) covers {referrers.nlargest(3).iloc[2] / len(df_ref) * 100:.2f}% of all non-na rows.')
mode_referrer = referrers.idxmax()

#check pct of rows we are filling in for
df_ref_na_vis_id_0 = df_proc[(df_proc['Referrer'].isna()) & (df_proc['Visitor Id']==0)]
pct_ref_na_vis_id_0 = len(df_ref_na_vis_id_0)/len(df_proc)*100
print(f"\n{pct_ref_na_vis_id_0:.2f}% of rows have a 'Visitor Id' == 0 and NaN 'Referrer'")

#since we are only affecting ~3% of the dataset, we can assign the mode 'Referrer' to the rows with NaN 'Referrer'
df_filtered = df_proc[df_proc['Referrer'].isna()]
vis_ids = set(df_proc[df_proc['Visitor Id'] != 0]['Visitor Id'])
locations = df_filtered['Location'].unique()
for location in locations:
    rand_int = np.random.randint(1400000000, 1600000000)
    while rand_int in vis_ids:
        rand_int = np.random.randint(1400000000, 1600000000)
    vis_ids.add(rand_int)

    # Assign Visitor Id and Referrer for the current location in df_filtered
    df_filtered.loc[df_filtered['Location'] == location, 'Visitor Id'] = rand_int
    df_filtered.loc[df_filtered['Location'] == location, 'Referrer'] = mode_referrer

# Update the original DataFrame with the changes
df_proc.update(df_filtered)

15.21% of rows in 'Landing Referrer' are na.
79.76% of visitors with no landing referrer were engaged in chat

The largest referrer (https://www.google.com/) covers 37.98% of all non-na rows.
The second-largest referrer (https://www.temosunrooms.com/) covers 8.93% of all non-na rows.
length of df_visitor_id_na: 57
length of df_refer_na: 55

The largest referrer (https://www.temosunrooms.com/) covers 17.78% of all non-na rows.
The second-largest referrer (https://www.temosunrooms.com/find-a-dealer/) covers 14.91% of all non-na rows.
The third-largest referrer (https://www.temosunrooms.com/contact/) covers 10.07% of all non-na rows.

3.20% of rows have a 'Visitor Id' == 0 and NaN 'Referrer'


**Step 5**: Generate Visitor ID's for chats without unique Visitor IDs

In [ ]:
#we will look into the remaining rows that have 'Visitor Id' == 0

df_proc[df_proc['Visitor Id']==0].head()

#these 5 chats have no relation to each other, as shown by differing date/location/lead_type
#we can just assign a unique visitor id

#update vis_ids
vis_ids = set(df_proc[df_proc['Visitor Id'] != 0]['Visitor Id'])

#assign a unique visitor id to each of the 5 chats without a visitor id
for index in df_proc[df_proc['Visitor Id'] == 0].index:
    rand_int = np.random.randint(1400000000, 1600000000)
    while rand_int in vis_ids:
        rand_int = np.random.randint(1400000000, 1600000000)

    df_proc.loc[index, 'Visitor Id'] = rand_int
    # Add the new visitor id to vis_ids to prevent duplication
    vis_ids.add(rand_int)

**Step 6**: Verify that all processed data was done correctly using assert statements

In [ ]:
#before analyzing the transcript text for sentiment, we will check if there are any rows without transcripts
df_proc[df_proc['Transcript Text'].isna()].head()

#clearly we cannot analyze the sentiment on these chats as they have no transcript, we will discard them
df_proc = df_proc.dropna(subset=['Transcript Text'])

assert not df_proc.isnull().values.any(), "There are missing values in the data."
assert not (df_proc['Visitor Id'] == 0).all(), "There are chats without a unique Visitor Id"
assert not (df_proc['Referrer'].isna()).any(), "There are missing Referrer values in the data."
assert ((df_proc['Lead Type Id'] >= 0) & (df_proc['Lead Type Id'] <= 3)).all(), "'Lead Type Id' values are not all between 0 and 3"
#there are no negative values in Sales

#assert not na_df.isnull().values.any(), "There are missing values in the data."
#there are no more missing values in the dataframe

#assert (na_df['Sales'] >= 0).all(), "'Sales' contains negative values."
#assert (na_df['Revenue'] >= 0).all(), "'Revenue' contains negative values."
#there are no negative values in Sales or Revenue

**Step 7**: Split 'Location' column into City, State, Country to break down into simpler data and possibly use for one-hot-encoding

In [ ]:
#since the 'Location' column contains a lot of info that could be difficult to parse, we need to separate it
#is follows the format
    # 'some_city, some_state, (some_country)'
#this can be done with regex

#regex pattern that reflects the one above
pattern = r'^([áA-Za-z\s.]+),( )?([A-Za-z0-9]{1,3}),( )?\(([A-Za-z]{2})\)$'

#rows where 'Location' does not match the pattern
invalid_locations = df_proc[~df_proc['Location'].str.match(pattern, na=False)]

#rows with invalid 'Location' values
print(invalid_locations['Location'].unique())
"""
['Unknown City, Unknown Region, (US)' 'Unknown City, Unknown Region, (CN)'
 'Unknown City, Unknown Region, (ES)' 'Unknown City, Unknown Region, (UA)'
 'Unknown City, Unknown Region, (IN)' 'Unknown City, Unknown Region, (GB)'
 'Unknown City, Unknown Region, (JP)']
"""

#fixing the Unkown Region problem
#pattern of the locations above
pattern1 = r'^([A-Za-z\s]+),\s*Unknown Region,\s*(\([A-Z]{2}\))$'

# Use `str.replace` with regex capturing groups to substitute "Unknown Region" with "UNK"
df_proc['Location'] = df_proc['Location'].str.replace(pattern1, r'\1, UNK, \2', regex=True)


#check again
pattern = r'^([áA-Za-z\s.]+),\s?([A-Za-z0-9]{1,3}),\s?\(([A-Za-z]{2})\)$'
invalid_locations = df_proc[~df_proc['Location'].str.match(pattern, na=False)]
print(invalid_locations['Location'].unique())
"""
[]
"""

#works!
#now we want to split the location into 3 columns City, State, Country
#this works easily because the regex
    # \1 is City
    # \2 is State
    # \3 is Country
# using 'str.extract' to create new columns based on the regex pattern
df_proc[['City', 'State', 'Country']] = df_proc['Location'].str.extract(pattern)

#checking if it worked
print(df_proc[['Location', 'City', 'State', 'Country']].head())

df_proc.drop(['Location'], axis=1, inplace=True)

#all good!
#print(df_proc.dtypes)

['Unknown City, Unknown Region, (US)' 'Unknown City, Unknown Region, (CN)'
 'Unknown City, Unknown Region, (ES)' 'Unknown City, Unknown Region, (UA)'
 'Unknown City, Unknown Region, (IN)' 'Unknown City, Unknown Region, (GB)'
 'Unknown City, Unknown Region, (JP)']
[]
                  Location          City State Country
0  Unknown City, UNK, (US)  Unknown City   UNK      US
1     Little Elm, TX, (US)    Little Elm    TX      US
2     Little Elm, TX, (US)    Little Elm    TX      US
3  Unknown City, UNK, (US)  Unknown City   UNK      US
4  Unknown City, UNK, (US)  Unknown City   UNK      US


**Step 8**: Convert date columns into datetime format

In [ ]:
# idea 1 implemented
# convert data colums to date time to allow easy calculation, providing format to avoid slow calculation
df_proc['Created On'] = pd.to_datetime(df_proc['Created On'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df_proc['Ended On'] = pd.to_datetime(df_proc['Ended On'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')


# calculate the duration in minutes
df_proc['Chat Duration'] = (df_proc['Ended On'] - df_proc['Created On']).dt.total_seconds()/60
df_proc.loc[df_proc['Chat Duration'] < 0, 'Chat Duration'] = 0

**Step 9**: Create a 1/0 numerical column detailing whether chat was closed due to inactivity or not

In [ ]:
# idea 2 implemented
# Update 'Inactivity' column based on the presence of the inactivity phrase in the transcript
inactivity_phrase = "This chatroom has been closed due to inactivity"
df_proc['Inactivity'] = df_proc['Transcript Text'].apply(lambda x: 1 if inactivity_phrase in x else 0)

**Step 10**: Create new dataframe with data about information given by visitor; 1/0 numerical true/false whether zip code, phone, email, or name was given using RegEx

In [ ]:
# idea 3 implemented
zip_code_pattern = r'\b\d{5}\b(-\d{4})?( \d{4})?'  # Matches a 5-digit zip code
phone_pattern = r'\b\d{3}[-. ]?\d{3}[-. ]?\d{4}\b'  # US phone format, e.g., 605-261-8272
email_pattern = r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'  # Standard email pattern
name_pattern = r'(((My name is |my name is |my names |my name\'s |name: |name is |name\'s |my name |fullname is |full name is )(([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)|([A-Z][a-z]+)|([A-Za-z]+\s[A-Za-z]+)))|((Visitor: )(([A-Z][a-z]+\s[A-Z][a-z]+)|([A-Za-z]+\s[A-Za-z]+\.))))' # name pattern following user text my name is

# generate one-hot encoded columns for customer information (as discussed)
df_customer_info_one_hot = pd.DataFrame({
    'Has Email': df_proc['Transcript Text'].apply(lambda x: 1 if re.search(email_pattern, x) else 0),
    'Has Phone Number': df_proc['Transcript Text'].apply(lambda x: 1 if re.search(phone_pattern, x) else 0),
    'Has Zip Code': df_proc['Transcript Text'].apply(lambda x: 1 if re.search(zip_code_pattern, x) else 0),
    'Has Name': df_proc['Transcript Text'].apply(lambda x: 1 if re.search(name_pattern, x) else 0),
})


# Check the updated DataFrame
print(df_customer_info_one_hot.head())

#ensure joining can occur by relating databases by IDs
df_customer_info_one_hot['Chat Id'] = df_proc['Chat Id']

   Has Email  Has Phone Number  Has Zip Code  Has Name
0          0                 0             0         1
1          1                 0             0         0
2          0                 0             0         0
3          1                 1             0         1
4          1                 1             1         1


## Data Processed

### **df_proc** is now processed with simplified columns without na/null/0 values
### **df_customer_info_one_hot** has data about the information given by the visitor

# II. Sentiment Analysis

a. Clean and process chatlog transcripts

b. Tokenize and Stem chatlog transcripts

c. Run VADER Sentiment Analysis

d. Hold sentiment analysis in **df_sentiment**

**Step 1**: Clean and process chatlog transcripts.

In [ ]:
# initialize VADER for further processing
sia = SentimentIntensityAnalyzer()
# initialize stemmer
stemmer = PorterStemmer()

# In this function I intend on cleaning the chat data.
# Since a chat might contain lots of extra data that us not useful and might
# even negatively influence the VADER analysis.
def clean_transcript(transcript):
    # remove timestamps (e.g., [1/2/2024 4:40:35 PM])
    transcript = re.sub(r"\[\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}:\d{2} [AP]M\]", "", transcript)
     # remove HTML garbage data
    transcript = re.sub(r'lt;.*?gt;', '', transcript)
    # remove extra whitespace
    transcript = re.sub(r"\s+", " ", transcript).strip()
    return transcript



     neg    neu    pos  compound
0  0.000  0.700  0.300    0.9565
1  0.000  0.823  0.177    0.9658
2  0.048  0.723  0.229    0.8053
3  0.000  0.823  0.177    0.9909
4  0.012  0.757  0.231    0.9894


**Step 2**: Tokenize and Stem Transcripts.

In [ ]:
def tokenize_transcript(transcript):
  # tokenize data into words
  tokens = word_tokenize(transcript)
  return tokens

def stem_transcript(tokens):
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return " ".join(stemmed_tokens)

**Step 3:** Run VADER Sentiment Analysis

In [ ]:
# function to analyze sentiment
def analyze_transcript_sentiment(transcript):
    cleaned_text = clean_transcript(transcript)
    tokenized_text = tokenize_transcript(cleaned_text)
    stemmed_text = stem_transcript(tokenized_text)
    sentiment = sia.polarity_scores(stemmed_text)
    return sentiment

**Step 4**: Store sentiment analysis in **df_sentiment**

In [ ]:
df_sentiment = df_proc['Transcript Text'].apply(analyze_transcript_sentiment).apply(pd.Series)
print(df_sentiment.head())

#ensure joining can occur by relating databases by IDs
df_sentiment['Chat Id'] = df_proc['Chat Id']


#lowest_index = df_sentiment['compound'].idxmin()
#lowest_sentiment_transcript = df_proc.iloc[lowest_index]['Transcript Text']
#print("Full Transcript with the Lowest Sentiment:")
#print(lowest_sentiment_transcript)
#print(sorted(df_sentiment['compound'])[1])

In [ ]:
#df_path = r'/home/apm204/cs210/Final Project/df_proc.csv'
#df_proc.to_csv(df_path, index=False)

#df_path = r'/home/apm204/cs210/Final Project/df_customer_info_one_hot.csv'
#df_customer_info_one_hot.to_csv(df_path, index=False)

#df_path = r'/home/apm204/cs210/Final Project/df_sentiment.csv'
#df_sentiment.to_csv(df_path, index=False)

#df_sentiment['Chat Id'] = df_proc['Chat Id']
#df_customer_info_one_hot['Chat Id'] = df_proc['Chat Id']

## Current Data Frames:
### 1) **df_proc:** The main data frame with the extracted data
### 2) **df_customer_info_one_hot** The data frame containing a one hot encoded breakdown of what chats provided personal infomation such as zip code, name, address, phone number.
### 3) **df_sentiment** The data frame contating the sentiment information as received from VADER

# III. Database

**Step 1**: Create smaller data frames to be turned into SQL tables

1. df_lead: Visitor referral history, mapping how they reached the TEMO Sunrooms Chat
2. df_location: Visitor location information grabbed from browser cookie data
3. df_chat: Chat information; length of chat, activity, whether the visitor engaged, and transcript
4. df_time: Time information; day of the week, month, weekend, chat duration, and whether the chat occurred in normal work hours

In [ ]:
df_lead = df_proc[['Chat Id','Referrer', 'Lead Id', 'Lead Type Id', 'Landing Referrer', 'Visitor Id']]
df_location = df_proc[['Chat Id','City', 'State', 'Country']]
df_chat = df_proc[['Chat Id','Chat Duration', 'Inactivity', 'visitor_engaged', 'Transcript Text']]

#df_path = r'/home/apm204/cs210/Final Project/df_lead.csv'
#df_lead.to_csv(df_path, index=False)
#df_path = r'/home/apm204/cs210/Final Project/df_location.csv'
#df_location.to_csv(df_path, index=False)
#df_path = r'/home/apm204/cs210/Final Project/df_chat.csv'
#df_chat.to_csv(df_path, index=False)

#'created_on' in datetime format
df_proc['Created On'] = pd.to_datetime(df_proc['Created On'], errors='coerce')
df_time = pd.DataFrame()

#extract main time features from 'created_on'
df_time['Chat Id'] = df_proc['Chat Id']
df_time['day_of_the_week'] = df_proc['Created On'].dt.day_name()  # Day of the week
df_time['month'] = df_proc['Created On'].dt.month_name()  # Month

#one hot encoded weekend, 1 for weekend and 0 for weekday
df_time['weekend'] = (df_proc['Created On'].dt.weekday >= 5).astype(int)
#pull over chat duration
df_time['chat_duration'] = df_proc['Chat Duration']

# Define work hours and days
def is_working_hours(datetime):
    # Check if day is Monday to Friday (0-4) and time is between 08:00 and 18:00
    return 0 <= datetime.weekday() <= 4 and datetime.hour >= 8 and datetime.hour < 18

df_time['work_hours'] = df_proc['Created On'].apply(lambda x: 1 if is_working_hours(x) else 0)

#print the dataframe to check
#print(df_time.head())

**Step 2**: Process all column names to be lowercase with '_' instead of spaces; write data frames to csv.

In [ ]:
def process_columns(df_var):
    cols = df_var.columns
    lower_cols = [col.lower() for col in cols]
    processed_cols = [col.replace(" ", "_") for col in lower_cols]
    return processed_cols

In [ ]:
df_chat.columns = process_columns(df_chat)
df_customer_info_one_hot.columns = process_columns(df_customer_info_one_hot)
df_lead.columns = process_columns(df_lead)
df_location.columns = process_columns(df_location)
df_sentiment.columns = process_columns(df_sentiment)
df_time.columns = process_columns(df_time)

print(df_chat.columns)
print(df_customer_info_one_hot.columns)
print(df_lead.columns)
print(df_location.columns)
print(df_sentiment.columns)
print(df_time.columns)

df_path = r'/home/apm204/cs210/Final Project/df_chat.csv'
df_chat.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_customer_info_one_hot.csv'
df_customer_info_one_hot.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_lead.csv'
df_lead.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_location.csv'
df_location.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_sentiment.csv'
df_sentiment.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_time.csv'
df_time.to_csv(df_path, index=False)

Index(['chat_id', 'chat_duration', 'inactivity', 'visitor_engaged',
       'transcript_text'],
      dtype='object')
Index(['has_email', 'has_phone_number', 'has_zip_code', 'has_name', 'chat_id'], dtype='object')
Index(['chat_id', 'referrer', 'lead_id', 'lead_type_id', 'landing_referrer',
       'visitor_id'],
      dtype='object')
Index(['chat_id', 'city', 'state', 'country'], dtype='object')
Index(['neg', 'neu', 'pos', 'compound', 'chat_id'], dtype='object')
Index(['chat_id', 'day_of_the_week', 'month', 'weekend', 'chat_duration',
       'work_hours'],
      dtype='object')


# SQL TABLE LIST

### 1) **df_lead:** Referrer information (access link and where they discovered chatbot), Visitor Id (check for repeats), Lead type (reason for chat)
### 2) **df_customer_info_one_hot:** The data frame containing a one hot encoded breakdown of what chats provided personal infomation such as zip code, name, email address, phone number.
### 3) **df_sentiment:** The data frame contating the sentiment information as received from VADER
### 4) **df_location:** location information including city, state, country (UNK for unknown)
### 5) **df_chat:** chat logistical information including time length of chat, use active/inactive, and the log of the chat
### 6) **df_time:** with day of the week, month, time of day to decide whether certain times of day have peaks in use retention and sentiment

**Step 3**: Create SQL Engine with 6 (unique) data frames, connected by 'chat_id' column

In [ ]:
#ls /home/apm204/cs210/Final\ Project/

#from sqlalchemy import create_engine, text
#create SQLite engine
#engine = create_engine('sqlite:////home/apm204/cs210/Final Project/chat_logs.db')
#with engine.connect() as conn:
    #conn.execute(text("CREATE TABLE IF NOT EXISTS test_table (id INTEGER PRIMARY KEY, name TEXT);"))
#with engine.connect() as conn:
    #result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    #print(result.fetchall())  # This should include 'test_table' if it was created successfully

#create a SQL Alchemy engine
engine = create_engine('sqlite:////home/apm204/cs210/Final Project/chat_logs.db')

#store each dataframe as a SQL table
df_lead.to_sql('df_lead', engine, index=False, if_exists='replace')
df_customer_info_one_hot.to_sql('df_customer_info_one_hot', engine, index=False, if_exists='replace')
df_sentiment.to_sql('df_sentiment', engine, index=False, if_exists='replace')
df_location.to_sql('df_location', engine, index=False, if_exists='replace')
df_chat.to_sql('df_chat', engine, index=False, if_exists='replace')
df_time.to_sql('df_time', engine, index=False, if_exists='replace')

#verify tables creation
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    print(result.fetchall())

[('test_table',), ('df_proc',), ('df_clusters2',), ('df_clusters',), ('df_lead',), ('df_customer_info_one_hot',), ('df_sentiment',), ('df_location',), ('df_chat',), ('df_time',)]


# IV. SQL Database Analysis

Use SQL Commands to find possible trends in data with joins and groups between different tables

(found trends for sentiment with respect to activity, lead type, and day of the week)

In [ ]:
#print('This query will help you determine if sentiment varies by day.')
#This query will help you determine if sentiment varies by day.

query = """
SELECT t.day_of_the_week, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_time t ON s.chat_id = t.chat_id
GROUP BY t.day_of_the_week;
"""
sentiment_day_of_week = pd.read_sql(query, engine)
#print(sentiment_day_of_week)


print('\n\nThis query checks if chats created during work hours (8am-6pm) have different sentiment')
#This query checks if chats created during work hours (8am-6pm) have different sentiment

query = """
SELECT t.work_hours, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_time t ON s.chat_id = t.chat_id
GROUP BY t.work_hours;
"""
sentiment_work_hours = pd.read_sql(query, engine)
print(sentiment_work_hours)


#print('\n\nThis query checks if longer chats have higher sentiment')
#This query checks if longer chats have higher sentiment

query = """
SELECT c.chat_duration, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_chat c ON s.chat_id = c.chat_id
GROUP BY c.chat_duration
ORDER BY c.chat_duration;
"""
sentiment_chat_duration = pd.read_sql(query, engine)
#print(sentiment_chat_duration)


print('\n\nSentiment Based on User Information (df_customer_info_one_hot)')
#Sentiment Based on User Information (df_customer_info_one_hot)

query = """
SELECT ci.has_email, ci.has_phone_number, ci.has_zip_code, ci.has_name, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_customer_info_one_hot ci ON s.chat_id = ci.chat_id
GROUP BY ci.has_email, ci.has_phone_number, ci.has_zip_code, ci.has_name;
"""
sentiment_customer_info = pd.read_sql(query, engine)
print(sentiment_customer_info)


#print('\n\nSentiment Based on Location (Country)')
#Sentiment Based on Location (Country)

query = """
SELECT l.country, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_location l ON s.chat_id = l.chat_id
GROUP BY l.country;
"""
sentiment_country = pd.read_sql(query, engine)
#print(sentiment_country)

### COULD BE GOOD
# english speaking vs non-english speaking


#print('\n\nSentiment Based on Location (State)')
#Sentiment Based on Location (State)

query = """
SELECT l.state, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_location l ON s.chat_id = l.chat_id
WHERE l.country = 'US'
GROUP BY l.state;
"""
sentiment_state = pd.read_sql(query, engine)
#print(sentiment_state)


print('\n\nSentiment Based on Referrer')
#Sentiment Based on Referrer

query = """
SELECT l.referrer, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_lead l ON s.chat_id = l.chat_id
GROUP BY l.referrer;
"""
sentiment_referrer = pd.read_sql(query, engine)
print(sentiment_referrer)

sentiment_referrer[sentiment_referrer['referrer']=='https://www.temosunrooms.com/']

len(df_lead[df_lead['referrer']=='https://www.temosunrooms.com/window-wall-system/'])

top_referrers = df_lead.groupby('referrer').size().sort_values(ascending=False).head(10)

print(top_referrers)

##could be a banger
# temosunrooms ads are working and bringing in the right people
# external links are negative leads


#print('\n\nSentiment for Returning Visitors (Duplicate Visitor IDs):')
#Sentiment for Returning Visitors (Duplicate Visitor IDs):
query = """
SELECT v.visitor_id, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_lead v ON s.chat_id = v.chat_id
GROUP BY v.visitor_id
HAVING COUNT(v.visitor_id) > 1;
"""
sentiment_returning_visitors = pd.read_sql(query, engine)
#print(sentiment_returning_visitors)

#sentiment_returning_visitors['avg_sentiment'].mean()


#print('\n\nSentiment based on Activity')
#Sentiment based on Activity

query = """
SELECT c.inactivity, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_chat c ON s.chat_id = c.chat_id
GROUP BY c.inactivity;
"""
sentiment_inactivity = pd.read_sql(query, engine)
#print(sentiment_inactivity)

#not bad, look for retention


print('\n\nSentiment based on Lead Type')
#Sentiment based on Lead Type

query = """
SELECT l.lead_type_id, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_lead l ON s.chat_id = l.chat_id
GROUP BY l.lead_type_id;
"""
sentiment_lead_type_id = pd.read_sql(query, engine)
print(sentiment_lead_type_id)

## shows a need to diagnose a customer inquiry to best help
## names are helpful but without properly diagnosing an issue first, sentiment significantly lacks

## sales (1) being the highest sentiment shows maybe where interests lie, money making over customer relations
## outsourced chatbot configuration may cause this as higher sales lead to greater reporting
## seems to have happened at the detriment of visitors that are not seeking sales


#print('\n\nSentiment for Returning Visitors (Duplicate Visitor IDs):')
#Sentiment for Returning Visitors (Duplicate Visitor IDs):

query = """
SELECT v.visitor_id, AVG(s.compound) as avg_sentiment
FROM df_sentiment s
JOIN df_lead v ON s.chat_id = v.chat_id
GROUP BY v.visitor_id
HAVING COUNT(v.visitor_id) > 1;
"""
sentiment_returning_visitors = pd.read_sql(query, engine)
#print(sentiment_returning_visitors)
#meh



This query checks if chats created during work hours (8am-6pm) have different sentiment
   work_hours  avg_sentiment
0           0       0.929859
1           1       0.942392


Sentiment Based on User Information (df_customer_info_one_hot)
    has_email  has_phone_number  has_zip_code  has_name  avg_sentiment
0           0                 0             0         0       0.867769
1           0                 0             0         1       0.947689
2           0                 0             1         0       0.940002
3           0                 0             1         1       0.975829
4           0                 1             0         0       0.936541
5           0                 1             0         1       0.950554
6           0                 1             1         0       0.971176
7           0                 1             1         1       0.981796
8           1                 0             0         0       0.953666
9           1                 0             0   

### What was found?

- Trend between sentiment compound score and referrer, non-TEMO links have lower sentiment

- Lead Type affects sentiment compound score as chats without a defined Lead Type have lower sentiment than those that do. Additionally, revenue generating avenues (Sales and Service leads) have a higher satisfaction [this can be due to training, visitor interest, targeting campaigns, etc.

- Weekdays during normal working hours leads to chats with higher sentiment

- The more information a user provides, the higher the sentiment [beneficial for user due to a better chat and TEMO Sunrooms due to user information]

In [ ]:
df_lead.loc[:, 'temo_landing_referrer'] = df_lead['landing_referrer'].str.contains('temo', case=False, na=False).astype(int)

#df_lead['google_landing_referrer'] = df_lead['landing_referrer'].str.contains('google', case=False, na=False).astype(int)
#df_lead['other_landing_referrer'] = (~df_lead['temo_landing_referrer'].astype(bool) & ~df_lead['google_landing_referrer'].astype(bool)).astype(int)
#df_lead['referrer_type'] =

df_customer_info_one_hot.loc[:, 'has_city'] = (df_location['city'] != 'Unknown City').astype(int)
df_customer_info_one_hot.loc[:, 'has_state'] = (df_location['state'] != 'UNK').astype(int)

#mapping dictionary for the days of the week
day_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}
#apply the mapping to the 'day_of_the_week' column
df_time['day_of_the_week_num'] = df_time['day_of_the_week'].map(day_mapping)

month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7
}
#apply the mapping to the 'month' column
df_time['month_num'] = df_time['month'].map(month_mapping)

/tmp/ipykernel_1773258/3644104793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lead.loc[:, 'temo_landing_referrer'] = df_lead['landing_referrer'].str.contains('temo', case=False, na=False).astype(int)


# V. Perform K-Means Clustering

In [ ]:
#select the relevant features for clustering
X = pd.concat([
    df_chat[['chat_duration', 'inactivity']],
    df_customer_info_one_hot[['has_email', 'has_phone_number', 'has_zip_code', 'has_name', 'has_city', 'has_state']],
    df_lead[['lead_type_id', 'temo_landing_referrer']],
    df_time[['day_of_the_week_num', 'month_num', 'weekend', 'work_hours']]
], axis=1)

df_proc['compound'] = df_sentiment['compound']
X['compound'] = df_sentiment['compound']  # Add sentiment score

#scale the features (important for K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#apply K-Means clustering (let's assume k=3 for this example)
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(X_scaled)

#add the cluster labels to the dataframe
df_proc['cluster'] = kmeans.labels_

#analyze the clusters (e.g., find out which features drive sentiment in each cluster)
#look at the cluster centers to see the average values of the features for each cluster
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)
print(cluster_centers)

#visualize the clusters (2D using PCA for simplicity)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

cluster_palette = {
    0: '#FF7F7F',  # Soft Red
    1: '#7FB3FF',  # Soft Blue
    2: '#7FFF7F',  # Soft Green
    3: '#FFBF7F',  # Soft Orange
    4: '#BF7FFF',  # Soft Purple
    5: '#FF7FBF'   # Soft Pink
}



#plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df_proc['cluster'], palette=cluster_palette, s=100)
plt.title('Customer Segmentation Using K-Means Clustering')
plt.show()

#plt.figure(figsize=(10, 6))
#sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df_proc['cluster'], palette=cluster_palette, s=100)
#plt.title('Customer Segmentation Using K-Means Clustering')
#visualization_path = r'cluster_visualization.png'
#plt.savefig(visualization_path)

#analyze clusters based on sentiment
cluster_sentiment = df_proc.groupby('cluster')['compound'].mean()
print(cluster_sentiment)

#df_customer_info_one_hot.columns

#df_path = r'/home/apm204/cs210/Final Project/df_proc.csv'
#df_proc.to_csv(df_path, index=False)

**# of Clusters: 5**

Used 5 clusters as it ended up yielding the most information in developing a TEMO Sunrooms ideal customer along with providing information about the most important avenues of capturing leads.

Will now analyze information about these clusters

**Substep**: Create a combined_df in order to succinctly visualize trends in data

In [ ]:
# List of DataFrames
dfs = [df_time, df_chat, df_customer_info_one_hot, df_lead, df_location, df_sentiment, df_proc]

# Concatenate DataFrames along columns (axis=1)
combined_df = pd.concat(dfs, axis=1)

#df_path = r'/home/apm204/cs210/Final Project/combined_df.csv'
#combined_df.to_csv(df_path, index=False)

#combined_df.columns
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

combined_df['total_info'] = combined_df['has_email'] + combined_df['has_phone_number'] + combined_df['has_zip_code'] + combined_df['has_name']

#cluster_groups = combined_df.groupby('cluster')
#print(cluster_groups.get_group(2))

df_clusters = pd.DataFrame()
df_clusters['chat_id'] = df_lead['chat_id']
df_clusters['clusters'] = kmeans.labels_

df_clusters.to_sql('df_clusters', engine, index=False, if_exists='replace')

1620

In [ ]:
#plot histograms for each feature (chat_duration, visitor_engaged, etc.) per cluster
features = ['chat_duration', 'inactivity', 'has_email', 'has_phone_number', 'has_zip_code', 'has_name', 'lead_type_id', 'has_city', 'has_state', 'temo_landing_referrer', 'day_of_the_week_num', 'month_num', 'weekend', 'work_hours', 'total_info']
combined_df['unk_city'] = (combined_df['city'] == 'Unknown City').astype(int)

###look at combined df
###make new referrer column 0-temo;1-google;2-other
### check what the spread is
for feature in features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=combined_df['cluster'], y=combined_df[feature], palette='viridis')
    plt.title(f'Distribution of {feature} by Cluster')
    plt.show()


In [ ]:
#### BAD USELESS


subset_combined_df = combined_df.copy()
subset_combined_df = subset_combined_df[subset_combined_df['chat_duration'] <= 30]
subset_combined_df = subset_combined_df[subset_combined_df['compound'] >= 0.85]


# Create a scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=subset_combined_df, x='chat_duration', y='compound', hue='cluster', palette='Set1', s=100, edgecolor='black')

# Add labels and title
plt.title('Chat Duration vs Compound Score by Clusters', fontsize=14)
plt.xlabel('Chat Duration', fontsize=12)
plt.ylabel('Compound Score', fontsize=12)

# Show the plot
plt.legend(title='Clusters')
plt.show()

In [ ]:
#df_path = r'/home/apm204/cs210/Final Project/df_lead.csv'
#df_lead.to_csv(df_path, index=False)

combined_df['temo_landing_referrer'] = df_lead['temo_landing_referrer']
combined_df['google_landing_referrer'] = df_lead['google_landing_referrer']
combined_df['other_landing_referrer'] = df_lead['other_landing_referrer']


print(combined_df.groupby('cluster')['temo_landing_referrer'].mean())
print(combined_df.groupby('cluster')['google_landing_referrer'].mean())
print(combined_df.groupby('cluster')['other_landing_referrer'].mean())

# Plot histograms for each feature (chat_duration, visitor_engaged, etc.) per cluster
#features = ['temo_landing_referrer', 'google_landing_referrer', 'other_referrer']
#for feature in features:
    #plt.figure(figsize=(10, 6))
    #sns.scatterplot(x=combined_df['cluster'], y=df_lead[feature], palette='viridis')
    #plt.title(f'Distribution of {feature} by Cluster')
    #plt.show()


i like the 5 clusters (1&2 being the best) as they show that Google users are less likely to give info and lead to slightly worse chats whereas temo landing referrers are great. this is good i like this. i think we can make the following observations


- prioritizing temo landing referrer is important, google ads/seo/paidsearch helps bring people in but has less quality


- this goes with another chart (reading between the lines) as cluster 2 has more 2,3 lead types (other/service) whereas temo has 1,2 lead types (sales/service) which brings in more money. focusing on temo over google ads is more important and greater money maker


- prioritize short and sweet chats as prolonged chat life kills sentiment value, make sure the user is engaged and responded to as quick as possible


- prioritize phone numbers and zip codes, cluster 1 has greatest sentiment and generally has zip and phone numbers --- knowing the area of individual as opposed to just their name gives TEMO a better understanding of their userbase. having a phone helps with this geolocational knowledge and provides a personal avenue to speak human-to-human with customers. this makes sense since the number of scams and phishing that occurs with names and emails make these things seem far more distant and less personal. calling a customer and understanding the region they live in (helped by phone whisper and branding) leads to greater sales.

In [ ]:
df_path = r'/home/apm204/cs210/Final Project/df_chat.csv'
df_chat.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_customer_info_one_hot.csv'
df_customer_info_one_hot.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_lead.csv'
df_lead.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_location.csv'
df_location.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_sentiment.csv'
df_sentiment.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_time.csv'
df_time.to_csv(df_path, index=False)
df_path = r'/home/apm204/cs210/Final Project/df_clusters.csv'
df_clusters.to_csv(df_path, index=False)

In [ ]:
print(len(df_raw))
print(len(combined_df))

1624
1620


In [ ]:
len(combined_df.columns)

46